# Image Classification with Pre-trained ResNet-50 using TensorFlow

In this JupyterLab notebook, we will explore the process of training an image classification model using the powerful ResNet-50 architecture. ResNet-50 is a deep convolutional neural network known for its exceptional performance in various computer vision tasks. Our implementation will utilize the TensorFlow library, which offers a flexible platform for building and training neural networks.

### Objective
The primary goal of this notebook is to train an image classification model that can categorize images from a given dataset into distinct classes.

### Key Steps
1. **Data Preparation:** We will start by preparing our dataset for both training and validation. This involves using the `ImageDataGenerator` class, which efficiently loads, preprocesses, and augments image data.

2. **Loading Pre-trained ResNet-50:** To leverage the power of transfer learning, we will load a pre-trained ResNet-50 model using TensorFlow's `keras.applications` module. This pre-trained model has already learned valuable feature representations from a large dataset (ImageNet).

3. **Custom Classification Layers:** Building upon the pre-trained ResNet-50 model, we will add custom classification layers. These layers will adapt the model to our specific image classification task.

4. **Model Compilation:** We will compile the model by specifying the optimizer, loss function, and evaluation metrics. This step prepares the model for training.

5. **Training:** Our next step involves training the model using the prepared training data. During training, the model's parameters will be fine-tuned to improve classification accuracy.

6. **Model Evaluation:** After training is complete, we will assess the model's performance using various metrics and visualizations.

7. **Saving the Model:** Finally, we will save the trained model's weights and architecture to an HDF5 file. This allows us to reuse the model later for inference without needing to retrain it.


In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model

# Path to the model file
model_file_path = './animal_models/resnet_model_tf.h5'

# Check if model file already exists
if os.path.exists(model_file_path):
    # Load the existing model
    print("Loading existing model.")
    model = load_model(model_file_path)
else:
    # Load and preprocess data using ImageDataGenerator
    train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
    train_generator = train_datagen.flow_from_directory(
        'dataset',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        'dataset',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )

    # Determine the number of classes based on the generator
    num_classes = len(train_generator.class_indices)

    # Load the pre-trained ResNet-50 model
    base_model = ResNet50(weights='imagenet', include_top=False)

    # Add custom classification layers on top of the pre-trained model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Training
    print("Training the model.")
    num_epochs = 10
    history = model.fit(train_generator, epochs=num_epochs, validation_data=validation_generator)

    # Save the trained model as an HDF5 file
    print("Saving the trained model.")
    os.makedirs('./animal_models', exist_ok=True)
    model.save(model_file_path)

print("Model is ready for inference.")

Loading existing model.
Model is ready for inference.
